In [96]:
with open('/Users/satyajitgupte/Downloads/input.txt', 'r', encoding='utf-8') as infile:
    text = infile.read()

In [97]:
print(f'Num Chars in Text = {len(text)}')

Num Chars in Text = 1115394


In [98]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [99]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [100]:
vocab_size

65

In [101]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [102]:
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda i: ''.join([itos[ix] for ix in i])

In [103]:
encode('hi there')

[46, 47, 1, 58, 46, 43, 56, 43]

In [104]:
print(decode(encode('hi there')))

hi there


In [105]:
import torch

In [106]:
# encode text
data = torch.tensor(encode(text), dtype=torch.long)

In [107]:
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [108]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [109]:
# split into train and val
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [110]:
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [111]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [112]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is {target}')


when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [113]:
torch.manual_seed(1337)
batch_size = 32

In [117]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - batch_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+1+block_size] for i in ix])
    return x, y

In [118]:
xb, yb = get_batch('train')

In [116]:
print("inputs = ", xb.shape,"\n", xb )

inputs =  torch.Size([32, 32]) 
 tensor([[58, 53,  1,  ..., 43, 52,  1],
        [49,  1, 39,  ..., 46, 43,  1],
        [59, 50, 42,  ...,  0, 35, 43],
        ...,
        [17, 24, 27,  ..., 47, 51,  1],
        [ 5, 57,  1,  ..., 54, 56, 53],
        [46,  1, 39,  ..., 53, 59,  1]])


In [86]:
print("outputs = ", yb.shape,"\n", yb )

outputs =  torch.Size([32, 32]) 
 tensor([[ 1, 54, 56,  ..., 50, 43, 57],
        [43, 39, 52,  ..., 53, 60, 43],
        [ 1, 47, 57,  ..., 53, 59, 57],
        ...,
        [ 1, 40, 43,  ..., 46, 43,  1],
        [ 0, 32, 46,  ...,  1, 40, 43],
        [41, 43,  6,  ..., 43, 56,  8]])


In [87]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} target is {target}')
            

when input is [43] target is 1
when input is [43, 1] target is 54
when input is [43, 1, 54] target is 56
when input is [43, 1, 54, 56] target is 47
when input is [43, 1, 54, 56, 47] target is 52
when input is [43, 1, 54, 56, 47, 52] target is 41
when input is [43, 1, 54, 56, 47, 52, 41] target is 43
when input is [43, 1, 54, 56, 47, 52, 41, 43] target is 5
when input is [51] target is 43
when input is [51, 43] target is 39
when input is [51, 43, 39] target is 52
when input is [51, 43, 39, 52] target is 57
when input is [51, 43, 39, 52, 57] target is 1
when input is [51, 43, 39, 52, 57, 1] target is 58
when input is [51, 43, 39, 52, 57, 1, 58] target is 53
when input is [51, 43, 39, 52, 57, 1, 58, 53] target is 1
when input is [46] target is 1
when input is [46, 1] target is 47
when input is [46, 1, 47] target is 57
when input is [46, 1, 47, 57] target is 1
when input is [46, 1, 47, 57, 1] target is 58
when input is [46, 1, 47, 57, 1, 58] target is 46
when input is [46, 1, 47, 57, 1, 58

In [88]:
xb

tensor([[43,  1, 54,  ...,  1, 50, 43],
        [51, 43, 39,  ..., 50, 53, 60],
        [46,  1, 47,  ..., 46, 53, 59],
        ...,
        [53,  1, 40,  ..., 32, 46, 43],
        [10,  0, 32,  ..., 58,  1, 40],
        [52, 41, 43,  ..., 60, 43, 56]])

In [89]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensors of ints
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = self(idx) # gives (B,T,C)
            logits = logits[:,-1, :] # we care only about the last time step, we get (B, T, C) --> (B,C)
            probs = F.softmax(logits, dim=-1) # last dim
            next_idx = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_idx), dim=1)
        return idx


In [90]:
model  = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)

In [91]:
print(decode(model.generate(torch.tensor(torch.zeros((1,1)), dtype=torch.long), 100)[0].tolist()))


egbeAhI$pbFPFVYdi'OPS;M 'yZ!bkownZPW:owLduQkUJ-!-QPqvCgJ'!!SS-F ELAZjUW?VA'WBs'ITbHnwcJLLWaW-Vs,&:bs


/var/folders/v3/cchdhwjd2p550zspwxwj6jlw0000gn/T/ipykernel_96749/2939767912.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(decode(model.generate(torch.tensor(torch.zeros((1,1)), dtype=torch.long), 100)[0].tolist()))


In [130]:
lr = 1e-3
max_steps = 100000
optimizer = torch.optim.AdamW(model.parameters(), lr)
batch_size = 32

for i in range(max_steps):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
    

2.42185640335083


In [133]:
print(decode(model.generate(torch.tensor(torch.zeros((1,1)), dtype=torch.long), 100)[0].tolist()))


LAn ormay ucearant hesis bll.
hequpr t outhyrat'd.
IIUSI atild adount.

Figequ, diente.

Wherann th 


/var/folders/v3/cchdhwjd2p550zspwxwj6jlw0000gn/T/ipykernel_96749/2939767912.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(decode(model.generate(torch.tensor(torch.zeros((1,1)), dtype=torch.long), 100)[0].tolist()))


### Attention - the mathematical trick

In [176]:
# matrix multiplication can be used to get weighted aggregation
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
# 

In [182]:
print("a -> \n", a)
print("b -> \n", b)
print("a @ b -> \n", a @ b)

a -> 
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b -> 
 tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
a @ b -> 
 tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [183]:
# 
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)

In [184]:
x

tensor([[[-0.0431, -1.6047],
         [ 1.7878, -0.4780],
         [-0.2429, -0.9342],
         [-0.2483, -1.2082],
         [-0.7688,  0.7624],
         [-1.5673, -0.2394],
         [ 2.3228, -0.9634],
         [ 2.0024,  0.4664]],

        [[ 0.8008,  1.6806],
         [ 0.3559, -0.6866],
         [-0.4934,  0.2415],
         [-1.1109,  0.0915],
         [-0.2516,  0.8599],
         [-0.3097, -0.3957],
         [ 0.8034, -0.6216],
         [-0.5920, -0.0631]],

        [[ 0.3057, -0.7746],
         [ 0.0349,  0.3211],
         [ 1.5736, -0.8455],
         [ 1.3123,  0.6872],
         [-1.2347, -0.4879],
         [-1.4181,  0.8963],
         [ 0.0499,  2.2667],
         [ 1.1790, -0.4345]],

        [[-0.8140, -0.7360],
         [-0.8371, -0.9224],
         [ 1.8113,  0.1606],
         [ 0.3672,  0.1754],
         [-1.1845,  1.3835],
         [-1.2024,  0.7078],
         [-1.0759,  0.5357],
         [ 1.1754,  0.5612]]])

In [188]:
# we want x[b,t] = mean_{i<=t} xb[b,i]
# version 1 - naive mean
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b, t] = torch.mean(xprev, dim=0)

In [206]:
# version 2 - using matrix multiplication for weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei/wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) -> (B, T, C) batched matrix multiply
torch.allclose(xbow, xbow2)

True

In [220]:
# version 3 - using softmax
tril = torch.tril(torch.ones(T, T))
wei = tril.masked_fill(tril == 0, float('-inf'))
wei = wei.softmax(dim=-1)
xbow3 = wei @ x
torch.allclose(xbow3, xbow)

True

In [245]:
# version 4 - self attention
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head performing self attention
head_size = 16
key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) -> (B,T,T)
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = wei.softmax(dim=1)

v = value(x)
out = wei @ v

#out 

In [247]:
out.shape

torch.Size([4, 8, 16])

In [249]:
n_embed

NameError: name 'n_embed' is not defined

In [234]:
k.transpose(-1, -2).shape

torch.Size([4, 16, 8])

In [285]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - batch_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+1+block_size] for i in ix])
    return x, y

@torch.no_grad()
def evaluate_loss():
    gpt.eval()
    out = {}
    for split in ["train", "test"]:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = gpt(xb, yb)
            losses[i] = loss.item()
        out[split] = losses.mean()
    gpt.train()
    return out
            
    

In [322]:
n_embd = 64 # token and position embedding size
block_size = 32 # context length
dropout = 0.0
n_layer = 4 # number of transformer blocks
device = 'cpu'
n_head = 4
eval_iters = 200

class Head(nn.Module):
    """one head of self attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute affinities (attention)
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T,C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei) # (B,T,T)
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out
    
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parrallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concat across last dim
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)
    
    
class Block(nn.Module):
    """ Transoformer block - communication followed by computation """
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
        

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb #(B,T,C)
        x = self.blocks(x) #(B,T,C)
        x = self.ln_f(x) #(B,T,C)
        logits = self.lm_head(x) #(B,T, vocab_size)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_tokens):
        # idx is (B,T)
        for _ in range(max_tokens):
            idx_context = idx[:,-block_size:] # crop context to block_size
            logits, _ = self(idx_context)
            # consider only the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            probs = F.softmax(logits, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx, next_idx), dim=1)
        return idx
        
        
        

In [333]:
gpt = GPTLanguageModel()
gpt = gpt.to(device)
lr = 1e-3
max_steps = 100000
optimizer = torch.optim.AdamW(gpt.parameters(), lr)
batch_size = 32
lossi = []

for i in range(max_steps):
    xb, yb = get_batch('train')
    logits, loss = gpt(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if i % 1000== 0:
        losses = evaluate_loss()
        print("\nIter={} train loss={:2f} val loss={:2f}".format(i, losses['train'], losses['test']))
        # sample from the model
        #print(decode(gpt.generate(torch.zeros((1,1)).long(), 100).tolist()[0]))
    lossi.append(loss.item())

#print(loss.item())
    


Iter=0 train loss=4.180072 val loss=4.174206

Iter=1000 train loss=1.957292 val loss=2.022863

Iter=2000 train loss=1.737550 val loss=1.893850

Iter=3000 train loss=1.647651 val loss=1.821699

Iter=4000 train loss=1.604235 val loss=1.776476

Iter=5000 train loss=1.572132 val loss=1.738808

Iter=6000 train loss=1.547580 val loss=1.715767

Iter=7000 train loss=1.523399 val loss=1.704546

Iter=8000 train loss=1.511895 val loss=1.693417


KeyboardInterrupt: 

In [334]:
# generate samples
print(decode(gpt.generate(torch.zeros((1,1)).long(), 1000).tolist()[0]))


First Clifford, In upon Tybalter fly.
My eyesick of York must would no hand!
Weepons these clameling, from so much I have that me ghost call'nger thy king them hearts they be affliction. Well thee woman,
Well me grow with him of his know, Come, for woman, it whom I lest,
And as in a state night, look how should with be thou hast unreason so is our that seeks the so request, her an merry shake power dies then?

QUEEN MAPERLARET:
Can Est he claughter'd me to be, that he count
and fairewelow't own while, master good to drown;
For so she we, not throng are do Romeo,
And where how I am little but nor the blow:
Ay, feect than userried of his knows in a maid with he thank
That it all opense her farls you frown.

ABREKEL:
Sir, Polining Of EDWARD:
Your love but when I, queen thy royal,
The reepInect and warranced upon lady shy stand
To where your clack his slip.

GLORDIVERS:
Ay, know, and and rubour Mainton,
And or I who duke drests never you,
Sir, and my swear Romeo,? and proads,
Let esment-n

In [299]:
x = torch.rand((3,10))

In [304]:
x[:,-8:]

tensor([[0.2081, 0.1857, 0.1532, 0.7946, 0.0362, 0.7352, 0.0390, 0.6962],
        [0.9008, 0.1008, 0.7824, 0.2185, 0.4749, 0.9446, 0.0310, 0.4892],
        [0.5252, 0.9241, 0.9331, 0.2598, 0.6783, 0.3642, 0.3723, 0.3621]])

In [303]:
x[:,:8]

tensor([[0.4978, 0.8541, 0.2081, 0.1857, 0.1532, 0.7946, 0.0362, 0.7352],
        [0.0286, 0.9920, 0.9008, 0.1008, 0.7824, 0.2185, 0.4749, 0.9446],
        [0.1533, 0.2051, 0.5252, 0.9241, 0.9331, 0.2598, 0.6783, 0.3642]])

In [305]:
x

tensor([[0.4978, 0.8541, 0.2081, 0.1857, 0.1532, 0.7946, 0.0362, 0.7352, 0.0390,
         0.6962],
        [0.0286, 0.9920, 0.9008, 0.1008, 0.7824, 0.2185, 0.4749, 0.9446, 0.0310,
         0.4892],
        [0.1533, 0.2051, 0.5252, 0.9241, 0.9331, 0.2598, 0.6783, 0.3642, 0.3723,
         0.3621]])